#Vedic sequence analyzer based on Rachel/Jeremy's awesome [Code-First Intro to Natural Language Processing](https://www.youtube.com/playlist?list=PL8l5P33wvCvIqO3UyvdkeRMoGg--VBvho) and respective [Notebook](https://github.com/fastai/course-nlp/) <a href="https://colab.research.google.com/github/animesh/notebooks/blob/master/vedSeq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai import *
from fastai.text import *

ModuleNotFoundError: No module named 'torch._C'

In [2]:
#bs=12
bs=48
#bs=128

In [3]:
#torch.cuda.set_device(0)

In [4]:
data_path = Config.data_path()
print(data_path)

/home/animeshs/.fastai/data


This will create a `viwiki` folder, containing a `viwiki` text file with the wikipedia contents. (For other languages, replace `vi` with the appropriate code from the [list of wikipedias](https://meta.wikimedia.org/wiki/List_of_Wikipedias).)

In [5]:
lang = 'sa'
#lang = 'vi'
# lang = 'zh'

In [6]:
name = f'{lang}wiki'
path = data_path/name
path.mkdir(exist_ok=True, parents=True)
lm_fns = [f'{lang}_wt', f'{lang}_wt_vocab']
print(name,lm_fns,path)

sawiki ['sa_wt', 'sa_wt_vocab'] /home/animeshs/.fastai/data/sawiki


In [7]:
#https://github.com/fastai/course-nlp/blob/master/nlputils.py
from nlputils import split_wiki,get_wiki

ImportError: cannot import name 'split_wiki' from 'nlputils' (/home/animeshs/anaconda3/lib/python3.7/site-packages/nlputils/__init__.py)

### Download data 
https://stackoverflow.com/a/31220935/1137129

In [10]:
import urllib.request as urllib2
from bs4 import *
from urllib.parse  import urljoin


def crawl(pages, depth=None):
    indexed_url = [] # a list for the main and sub-HTML websites in the main website
    for i in range(depth):
        for page in pages:
            if page not in indexed_url:
                indexed_url.append(page)
                try:
                    c = urllib2.urlopen(page)
                except:
                    print( "Could not open %s" % page)
                    continue
                soup = BeautifulSoup(c.read())
                links = soup('a') #finding all the sub_links
                for link in links:
                    if 'href' in dict(link.attrs):
                        url = urljoin(page, link['href'])
                        if url.find("'") != -1:
                                continue
                        url = url.split('#')[0] 
                        if url[0:4] == 'http':
                                indexed_url.append(url)
        pages = indexed_url
    return indexed_url

In [11]:
pagelist=["https://sa.wikisource.org/wiki/%E0%A4%8B%E0%A4%97%E0%A5%8D%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%83"]
urls = crawl(pagelist, depth=1)
print( urls )

['https://sa.wikisource.org/wiki/%E0%A4%8B%E0%A4%97%E0%A5%8D%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%83', 'https://sa.wikisource.org/wiki/%E0%A4%8B%E0%A4%97%E0%A5%8D%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%83', 'https://sa.wikisource.org/wiki/%E0%A4%8B%E0%A4%97%E0%A5%8D%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%83', 'https://sa.wikisource.org/wiki/%E0%A4%B8%E0%A4%9E%E0%A5%8D%E0%A4%9A%E0%A4%BF%E0%A4%95%E0%A4%BE:Goddess_Ganga._Delhi_National_Museum_ni01-11.jpg', 'https://sa.wikisource.org/wiki/%E0%A4%B8%E0%A4%9E%E0%A5%8D%E0%A4%9A%E0%A4%BF%E0%A4%95%E0%A4%BE:Goddess_Ganga._Delhi_National_Museum_ni01-11.jpg', 'https://sa.wikisource.org/wiki/%E0%A4%B8%E0%A5%8D%E0%A4%95%E0%A4%A8%E0%A5%8D%E0%A4%A6%E0%A4%AA%E0%A5%81%E0%A4%B0%E0%A4%BE%E0%A4%A3%E0%A4%AE%E0%A5%8D/%E0%A4%96%E0%A4%A3%E0%A5%8D%E0%A4%A1%E0%A4%83_%E0%A5%AA_(%E0%A4%95%E0%A4%BE%E0%A4%B6%E0%A5%80%E0%A4%96%E0%A4%A3%E0%A5%8D%E0%A4%A1%E0%A4%83)/%E0%A4%85%E0%A4%A7%E0%A5%8D%E0%A4%AF%E0%A4%BE%E0%A4%AF%E0%A4%83_%E0%A5%A6%E0%A5%AF%E0%A5%A8', 'https://sa.wikisource.org/

In [29]:
pagelist=["https://en.wikisource.org/wiki/The_Rig_Veda"]
urls = crawl(pagelist, depth=1)
print( urls )

['https://en.wikisource.org/wiki/The_Rig_Veda', 'https://en.wikisource.org/wiki/The_Rig_Veda', 'https://en.wikisource.org/wiki/The_Rig_Veda', 'https://en.wikisource.org/wiki/File:Ambox_style_icon.svg', 'https://en.wikisource.org/wiki/Index:The_Hymns_of_the_Rigveda_Vol_1.djvu', 'https://en.wikisource.org/wiki/Index:The_Hymns_of_the_Rigveda_Vol_2.djvu', 'https://en.wikisource.org/wiki/Help:Match_and_Split', 'https://en.wikisource.org/wiki/Help:Proofread', 'https://en.wikisource.org/wiki/Author:Ralph_T._H._Griffith', 'https://en.wikisource.org/wiki/Portal:Portals', 'https://en.wikisource.org/wiki/Portal:Portals', 'https://en.wikisource.org/wiki/Portal:Religion', 'https://en.wikisource.org/wiki/Special:SiteMatrix', 'https://en.wikisource.org/wiki/Special:SiteMatrix', 'https://en.wikipedia.org/wiki/Rigveda', 'https://commons.wikimedia.org/wiki/Category:Rig_Veda', 'https://en.wikiquote.org/wiki/Rigveda', 'https://www.wikidata.org/wiki/Q727413', 'https://en.wikisource.org/wiki/The_Vedas', 'ht

In [30]:
def parseWWW(page):
    c = urllib2.urlopen(page)
    soup = BeautifulSoup(c.read())
    return soup.get_text() #finding all the text

In [43]:
text = parseWWW(urls[90])
text = parseWWW('https://sa.wikisource.org/wiki/%E0%A4%8B%E0%A4%97%E0%A5%8D%E0%A4%B5%E0%A5%87%E0%A4%A6%E0%A4%83')
print(text)
#print(text.encode('utf-8'))




ऋग्वेदः - विकिस्रोतः
document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["0\t1\t2\t3\t4\t5\t6\t7\t8\t9","०\t१\t२\t३\t४\t५\t६\t७\t८\t९"],"wgDefaultDateFormat":"dmy","wgMonthNames":["","जनवरी","फरवरी","मार्च","अप्रैल","मई","जून","जुलाई","अगस्त","सितम्बर","अक्तूबर","नवम्बर","दिसम्बर"],"wgMonthNamesShort":["","जेन.","फेब.","मार्च.","एप्र॰","मे","जून","जुल.","ओग.","सेप्.","ओक्टो.","नवे.","डिस."],"wgRequestId":"XfOY-ApAICsAACOFXHEAAADS","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"ऋग्वेदः","wgTitle":"ऋग्वेदः","wgCurRevisionId":214629,"wgRevisionId":214629,"wgArticleId":27,"wgIsArticle":!0,"wgIsRedirect":
!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["ऋग्वेदः"],"wgPageContentLanguage":"sa","wgPageContentModel":"wikitext","wgRelevantPageName":"ऋग्वेदः","wgRelevantArticleId":27,"wgIsProbablyEditable":!0,"wgRelevantPa

In [45]:
with open(path / name, 'w') as fo:
    fo.write(text)

In [48]:
path / name / lang

PosixPath('/home/animeshs/.fastai/data/sawiki/sawiki/sa')

In [89]:
def split_wiki(path,name):
    dest = path / 'split.txt'
    print(dest)
    f = dest.open('w')
    name = name
    lines = (path/name).open()
    print(lines)
    for i,l in enumerate(lines):
        if i%1000 == 0: print(i)
        l=l.split(' ')
        #l=' '.join(l)
        f.write(str(l))
    f.close()
    return dest

This function splits the single wikipedia file into a separate file per article. This is often easier to work with.

In [90]:
dest = split_wiki(path,name)

/home/animeshs/.fastai/data/sawiki/split.txt
<_io.TextIOWrapper name='/home/animeshs/.fastai/data/sawiki/sawiki' mode='r' encoding='UTF-8'>
0


In [88]:
!head dest

head: cannot open 'dest' for reading: No such file or directory


### Create pretrained model

In [91]:
data = (TextList.from_folder(path)
            .split_by_rand_pct(0.1, seed=42)
            .label_for_lm()           
            .databunch(bs=bs, num_workers=1))

data.save(f'{lang}_databunch')
len(data.vocab.itos),len(data.train_ds)

/home/animeshs/anaconda3/lib/python3.7/site-packages/fastai/data_block.py:457: UserWarning: Your validation set is empty. If this is by design, use `split_none()`
                 or pass `ignore_empty=True` when labelling to remove this warning.
  or pass `ignore_empty=True` when labelling to remove this warning.""")


(80, 1)

In [92]:
print(path,f'{lang}_databunch')
#!mv /home/ash022/.fastai/data/peptidewiki/docs/peptide_databunch /home/ash022/.fastai/data/peptidewiki/peptide_databunch

/home/animeshs/.fastai/data/sawiki sa_databunch


In [96]:
learn = language_model_learner(data, AWD_LSTM, drop_mult=0.5, pretrained=False)#.to_fp16()

In [97]:
lr = 1e-2
lr *= bs/48  # Scale learning rate by batch size

In [98]:
learn.unfreeze()
learn.fit_one_cycle(10, lr, moms=(0.8,0.7))

Save the pretrained model and vocab:

In [99]:
mdl_path = path/'models'
mdl_path.mkdir(exist_ok=True)
learn.to_fp32().save(mdl_path/lm_fns[0], with_opt=False)
learn.data.vocab.save(mdl_path/(lm_fns[1] + '.pkl'))

In [ ]:
accuracy(preds_avg,targs_b),f1(preds_avg,targs_b)

## Future work
* Try [SentencePiece](https://github.com/google/sentencepiece)
* Check activation with [Interpretability Beyond Feature Attribution: Quantitative Testing with Concept Activation Vectors](https://github.com/tensorflow/tcav)
* [GPT2](https://gist.github.com/mohdsanadzakirizvi/f1419fba3907af2baf6e0f6ab2a53d5b)
* [Unsupervised word embeddings capture latent knowledge from materials science literature](https://www.nature.com/articles/s41586-019-1335-8)
